## Tutorial 2 : identical to tutorial 1, but with the DEBUG option turned on - the Cypher queries and the data binding managed by NeoAccess will become visible.

### CAUTION: only meant for people who are knowledgeable about Cypher, and want an under-the-hood look at the NeoAccess library!  

#### (`debug` mode ON)

#### [Overview and Intro article](https://julianspolymathexplorations.blogspot.com/2023/06/neo4j-python-neoaccess-library.html) to accompany these tutorials

In [1]:
import set_path      # Importing this module will add the project's home directory to sys.path

Added 'D:\Docs\- MY CODE\Brain Annex\BA-Win7' to sys.path


In [2]:
import os
import sys
import getpass

from neoaccess import NeoAccess

# Connect to the database

In [3]:
# Save your credentials here - or use the prompts given by the next cell
host = ""             # EXAMPLES:  bolt://123.456.789.012   OR   neo4j://localhost
password = ""

In [3]:
print("To create a database connection, enter the host IP, but leave out the port number: (EXAMPLES:  bolt://1.2.3.4  OR  neo4j://localhost )\n")

host = input("Enter host IP WITHOUT the port number.  EXAMPLE: bolt://123.456.789.012 ")
host += ":7687"    # EXAMPLE of host value:  "bolt://123.456.789.012:7687"

password = getpass.getpass("Enter the database password:")

print(f"\n=> Will be using: host='{host}', username='neo4j', password=**********")

To create a database connection, enter the host IP, but leave out the port number: (EXAMPLES:  bolt://1.2.3.4  OR  neo4j://localhost )



Enter host IP WITHOUT the port number.  EXAMPLE: bolt://123.456.789.012  bolt://155.248.202.124
Enter the database password: ········



=> Will be using: host='bolt://155.248.202.124:7687', username='neo4j', password=**********


In [4]:
db = NeoAccess(host=host,
               credentials=("neo4j", password), debug=True)   # Notice the debug option being ON

~~~~~~~~~ Initializing NeoAccess object ~~~~~~~~~
Attempting to connect to Neo4j host 'bolt://155.248.202.124:7687', with username 'neo4j'
Connection to host 'bolt://155.248.202.124:7687' established


In [5]:
print("Version of the Neo4j driver: ", db.version())

Version of the Neo4j driver:  4.4.11


# Examples of basic NeoAccess library operations

In [6]:
db.empty_dbase()       # WARNING: USE WITH CAUTION!!!

In [7]:
# Create a "Car" node and a "Person" node

neo_car = db.create_node("Car", {'color': 'white', 'make': 'Toyota'})  # create_node returns the internal database ID of the new node

neo_person = db.create_node("Person", {'name': 'Julian'})


In create_node().  Query:
    CREATE (n :`Car` {`color`: $par_1, `make`: $par_2}) RETURN n
Data binding:
    {'par_1': 'white', 'par_2': 'Toyota'}


In create_node().  Query:
    CREATE (n :`Person` {`name`: $par_1}) RETURN n
Data binding:
    {'par_1': 'Julian'}



In [8]:
# Link the "Car" node to the "Person" node (using internal database ID's to refer to existing nodes)
number_added = db.add_links(match_from=neo_car, match_to=neo_person, rel_name="OWNED_BY")  

number_added

In add_links()
    cypher_match_from: CYPHER-PROCESSED match structure:
    node: (from)    where: id(from) = 31    data_binding: {}    dummy_node_name: from
    cypher_match_to: CYPHER-PROCESSED match structure:
    node: (to)    where: id(to) = 1    data_binding: {}    dummy_node_name: to

In add_links().  Query:
    
            MATCH (from), (to)
            WHERE (id(from) = 31 AND id(to) = 1)
            MERGE (from) -[:`OWNED_BY`]-> (to)           
            

In update_query(). Attributes of ResultSummary object:
    metadata -> {'query': '\n            MATCH (from), (to)\n            WHERE (id(from) = 31 AND id(to) = 1)\n            MERGE (from) -[:`OWNED_BY`]-> (to)           \n            ', 'parameters': {}, 'server': <neo4j.api.ServerInfo object at 0x000000001A060880>, 't_first': 18, 'fields': [], 'bookmark': 'FB:kcwQxosFA855RdybwjuMf2O2J8oAAPzukA==', 'stats': {'relationships-created': 1}, 'type': 'w', 't_last': 0, 'db': 'neo4j', 'notifications': [{'severity': 'WARNING',

1

![Two nodes and a link](../BrainAnnex/docs/tutorials_1.png)

In [9]:
# Retrieve the car node (in the most straightforward way, using an internal database ID)
db.get_nodes(neo_car)

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 31    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n) WHERE (id(n) = 31) RETURN n



[{'color': 'white', 'make': 'Toyota'}]

In [10]:
# Retrieve a single property of the car node (to be used when only 1 node is present)
db.get_nodes(neo_car, single_cell="color")

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 31    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n) WHERE (id(n) = 31) RETURN n



'white'

In [11]:
# How many owners does the car have?
db.count_links(neo_car, rel_name="OWNED_BY", rel_dir="OUT") 

In count_links()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 31    data_binding: {}    dummy_node_name: n

In count_links().  Query:
    MATCH (n) - [:OWNED_BY] -> (neighbor )WHERE (id(n) = 31) RETURN count(neighbor) AS link_count



1

In [12]:
# Look up information about the car owner(s)
db.follow_links(neo_car, rel_name="OWNED_BY", rel_dir="OUT") 

In follow_links()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 31    data_binding: {}    dummy_node_name: n

In follow_links().  Query:
    MATCH (n) - [:OWNED_BY] -> (neighbor )WHERE (id(n) = 31) RETURN neighbor



[{'name': 'Julian'}]

#### Let's pretend we didn't save the internal database ID's of our 2 nodes; let's epecify those node, for later locating them

In [13]:
# Lets provide a way to later look up the "Car" node, using the match() method.
# IMPORTANT: NO DATABASE OPERATION IS ACTUALLY PERFORMED HERE!  We're just saving up all the specs 
#            (to indentify a node, OR GROUP OF NODES) into an object of class "NodeSpecs"

car_match = db.match(labels="Car", properties={'color': 'white', 'make': 'Toyota'})

car_match

In [14]:
print(car_match)    # Let's look at the specs we saved up; they will be used LATER in actual database operations

RAW match structure:
    internal_id: None    labels: Car    key_name: None    key_value: None    properties: {'color': 'white', 'make': 'Toyota'}    clause: None    clause_dummy_name: None


In [15]:
# A lot of parameters can be passed to match().  Some examples of alternative ways to specify the same node as above:

car_match_alt = db.match(labels="Car", clause="n.color = 'white' AND n.make = 'Toyota'", dummy_node_name="n")

print(car_match_alt)

RAW match structure:
    internal_id: None    labels: Car    key_name: None    key_value: None    properties: None    clause: n.color = 'white' AND n.make = 'Toyota'    clause_dummy_name: n


In [16]:
# Various ways to specify our Person node (again, NO DATABASE OPERATION IS ACTUALLY PERFORMED HERE!)

person_match  = db.match(labels="Person", properties={'name': 'Julian'})
person_match_alt_1 = db.match(labels="Person", clause="n.name = 'Julian'", dummy_node_name="n")
person_match_alt_2 = db.match(labels="Person", key_name='name', key_value='Julian')

#### Armed with the "NodeSpecs" objects, we can do all sort of operations - passing those objects (serving as "handles") in lieu of the internal database ID's that we lack (and which would require extra database operations to retrieve)
Note: NO EXTRA DATABASE OPERATIONS ARE WASTED ON LOCATING THE NODES!  Efficient, 1-step, database queries are created and executed at the very LAST stage; for example to create the following link

In [17]:
# Link the "Person" node to the "Car" node (a reverse link of the one we created before)
# HERE'S WHERE THE (SINGLE) DATABASE OPERATION ACTUALLY GETS PERFORMED
number_added = db.add_links(match_from=person_match, match_to=car_match, rel_name="OWNS")  

number_added

In add_links()
    cypher_match_from: CYPHER-PROCESSED match structure:
    node: (from :`Person` {`name`: $from_par_1})    where:     data_binding: {'from_par_1': 'Julian'}    dummy_node_name: from
    cypher_match_to: CYPHER-PROCESSED match structure:
    node: (to :`Car` {`color`: $to_par_1, `make`: $to_par_2})    where:     data_binding: {'to_par_1': 'white', 'to_par_2': 'Toyota'}    dummy_node_name: to

In add_links().  Query:
    
            MATCH (from :`Person` {`name`: $from_par_1}), (to :`Car` {`color`: $to_par_1, `make`: $to_par_2})
            
            MERGE (from) -[:`OWNS`]-> (to)           
            
Data binding:
    {'from_par_1': 'Julian', 'to_par_1': 'white', 'to_par_2': 'Toyota'}

In update_query(). Attributes of ResultSummary object:
    metadata -> {'query': '\n            MATCH (from :`Person` {`name`: $from_par_1}), (to :`Car` {`color`: $to_par_1, `make`: $to_par_2})\n            \n            MERGE (from) -[:`OWNS`]-> (to)           \n            ', 'pa

1

![Two nodes and a link](../BrainAnnex/docs/tutorials_2.png)

#### Some verifications

The "Car" node can be found and extracted (performing a DATABASE OPERATION), using EITHER its Internal Database ID (which we had saved at the very beginning, though we we were acting like we didn't) OR any of the alternative ways we created to specify it

In [18]:
db.get_nodes(neo_car)    # Fetch by the internal database ID

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 31    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n) WHERE (id(n) = 31) RETURN n



[{'color': 'white', 'make': 'Toyota'}]

In [19]:
db.get_nodes(car_match)   # Fetch by "NodeSpecs" object returned by the match() method

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n :`Car` {`color`: $n_par_1, `make`: $n_par_2})    where:     data_binding: {'n_par_1': 'white', 'n_par_2': 'Toyota'}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n :`Car` {`color`: $n_par_1, `make`: $n_par_2})  RETURN n
Data binding:
    {'n_par_1': 'white', 'n_par_2': 'Toyota'}



[{'color': 'white', 'make': 'Toyota'}]

In [20]:
db.get_nodes(car_match_alt)   # Fetch by an alternate version of the "NodeSpecs" object

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n :`Car` )    where: n.color = 'white' AND n.make = 'Toyota'    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n :`Car` ) WHERE (n.color = 'white' AND n.make = 'Toyota') RETURN n



[{'color': 'white', 'make': 'Toyota'}]

_Likewise for the "Person" node:_

In [21]:
db.get_nodes(neo_person)    # Fetch by the internal database ID

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n)    where: id(n) = 1    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n) WHERE (id(n) = 1) RETURN n



[{'name': 'Julian'}]

In [22]:
db.get_nodes(person_match)   # Fetch by "NodeSpecs" object returned by the match() method

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n :`Person` {`name`: $n_par_1})    where:     data_binding: {'n_par_1': 'Julian'}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n :`Person` {`name`: $n_par_1})  RETURN n
Data binding:
    {'n_par_1': 'Julian'}



[{'name': 'Julian'}]

In [23]:
db.get_nodes(person_match_alt_1)

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n :`Person` )    where: n.name = 'Julian'    data_binding: {}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n :`Person` ) WHERE (n.name = 'Julian') RETURN n



[{'name': 'Julian'}]

In [24]:
db.get_nodes(person_match_alt_2)

In get_nodes()
    match_structure: CYPHER-PROCESSED match structure:
    node: (n :`Person` {`name`: $n_par_1})    where:     data_binding: {'n_par_1': 'Julian'}    dummy_node_name: n

In get_nodes().  Query:
    MATCH (n :`Person` {`name`: $n_par_1})  RETURN n
Data binding:
    {'n_par_1': 'Julian'}



[{'name': 'Julian'}]

## If you know the Cypher query language, and simply want to run a generic query, no problem!

In [25]:
q = '''MATCH (p :Person) -[:OWNS] -> (c :Car) -[OWNED_BY] -> (p)
       RETURN p.name, c.color, c.make
    '''    # This query will verify the forward and reverse links that we created earlier

In [26]:
db.query(q)    # Run the query; by default, it will return a list of records (each record is a dict)

[{'p.name': 'Julian', 'c.color': 'white', 'c.make': 'Toyota'}]

In [27]:
q_paint_car_red = '''MATCH (c :Car) -[OWNED_BY] -> (p :Person {name: 'Julian'})
                     SET c.color = 'red'
                  '''    # Paint all of Julian's cars red!

result = db.update_query(q_paint_car_red)   # It returns a dict of info about what it did

result

In update_query(). Attributes of ResultSummary object:
    metadata -> {'query': "MATCH (c :Car) -[OWNED_BY] -> (p :Person {name: 'Julian'})\n                     SET c.color = 'red'\n                  ", 'parameters': {}, 'server': <neo4j.api.ServerInfo object at 0x000000001A060880>, 't_first': 0, 'fields': [], 'bookmark': 'FB:kcwQxosFA855RdybwjuMf2O2J8oAAPzwkA==', 'stats': {'properties-set': 1}, 'type': 'w', 't_last': 0, 'db': 'neo4j'}
    server -> <neo4j.api.ServerInfo object at 0x000000001A060880>
    database -> neo4j
    query -> MATCH (c :Car) -[OWNED_BY] -> (p :Person {name: 'Julian'})
                     SET c.color = 'red'
                  
    parameters -> {}
    query_type -> w
    plan -> None
    profile -> None
    notifications -> None
    counters -> {'properties_set': 1}
    result_available_after -> 0
    result_consumed_after -> 0


{'properties_set': 1, 'returned_data': []}

In [28]:
db.query(q)   # Re-run the earlier query (to verify the forward and reverse links); notice how the car is now red

[{'p.name': 'Julian', 'c.color': 'red', 'c.make': 'Toyota'}]